In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install bayesian-optimization==1.4.3

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
from bayes_opt import BayesianOptimization
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor

warnings.filterwarnings('ignore')


data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HackED_beta/US_TOP_100_stocks.csv')
data['date'] = pd.to_datetime(data['date'])

#bins = [-np.inf,-3, -1, -0.25, 0.25, 1, 3, np.inf]
#data['category'] = pd.cut(data['change'], bins=bins, labels=False)

train = data[#(merged_data['date'] >= pd.Timestamp(year=1990, month=1, day=1)) &
                    (data['date'] < pd.Timestamp(year=2022, month=1, day=1))]
test = data[data['date'] >= pd.Timestamp(year=2022, month=1, day=1)]


In [ ]:
print(train.columns)

Index(['date', 'open', 'high', 'low', 'close', 'adj close', 'volume', 'ticker',
       'day_of_week', 'rate', 'middle', 'tp', 'boll', 'boll_ub', 'boll_lb',
       'macd', 'macds', 'macdh', 'pvo', 'pvos', 'pvoh', 'ppo', 'ppos', 'ppoh',
       'qqe', 'qqel', 'qqes', 'cr', 'cr-ma1', 'cr-ma2', 'cr-ma3', 'tr', 'dx',
       'adx', 'adxr', 'log-ret', 'wt1', 'wt2', 'supertrend_ub',
       'supertrend_lb', 'supertrend', 'bop', 'cti', 'eribull', 'eribear',
       'rvgi', 'rvgis', 'kst', 'ao', 'aroon', 'atr', 'cci', 'change', 'chop',
       'cmo', 'coppock', 'dma', 'ichimoku', 'inertia', 'ftr', 'kama', 'kdjk',
       'kdjd', 'kdjj', 'ker', 'mfi', 'ndi', 'pdi', 'pgo', 'psl', 'rsi', 'rsv',
       'stochrsi', 'tema', 'trix', 'wr', 'vr', 'vwma'],
      dtype='object')


In [ ]:
feature_columns = ['Bullish', 'Neutral', 'Bearish', 'Bullish 8-week Mov Avg',
                   'Bull-Bear Spread', 'volume', 'day_of_week', 'boll', 'boll_ub', 'boll_lb',
                   'macd', 'macds', 'macdh', 'pvo', 'pvos', 'pvoh', 'ppo', 'ppos', 'ppoh',
                   'qqe', 'qqel', 'qqes', 'cr', 'cr-ma1', 'cr-ma2', 'cr-ma3', 'tr', 'dx',
                   'adx', 'adxr', 'log-ret', 'wt1', 'wt2', 'supertrend_ub', 'supertrend_lb',
                   'supertrend', 'bop', 'cti', 'eribull', 'eribear', 'rvgi', 'rvgis', 'kst',
                   'ao', 'aroon', 'atr', 'cci', 'change', 'chop', 'cmo', 'coppock', 'dma',
                   'ichimoku', 'inertia', 'ftr', 'kama', 'kdjk', 'kdjd', 'kdjj', 'ker',
                   'mfi', 'ndi', 'pdi', 'pgo', 'psl', 'rsi', 'rsv', 'stochrsi', 'tema',
                   'trix', 'wr', 'vr', 'vwma', 'vix_close', 'vix_open']

for dataset in [train, test]:
    for col in feature_columns:
        for shift_day in range(1, 30):
            dataset[f'{col}_prev_{shift_day}'] = dataset[col].shift(shift_day)



    #for shift_day in range(1, 61):
        #dataset[f'change_prev_{shift_day}'] = dataset['change'].shift(shift_day)


scaler = StandardScaler()
#list_to_add = ['day_of_week', 'vix_Open']

shifted_feature_columns = [f'{col}_prev_{shift_day}' for col in feature_columns for shift_day in range(1, 30)]
#shifted_change_columns = [f'change_prev_{shift_day}' for shift_day in range(1, 61)]
all_shifted_columns = shifted_feature_columns #+ shifted_change_columns #+ list_to_add

X_train = scaler.fit_transform(train[all_shifted_columns])
X_test = scaler.transform(test[all_shifted_columns])

y_train = train['change']
y_test = test['change']

NameError: ignored

In [ ]:
print(all_shifted_columns)
print(train.shape)

In [ ]:
def optimize_model(model_eval, pbounds, init_points=5, n_iter=10):
    optimizer = BayesianOptimization(f=model_eval, pbounds=pbounds, random_state=42)
    optimizer.maximize(init_points=init_points, n_iter=n_iter)
    best_params = optimizer.max['params']
    for key, val in best_params.items():
        if isinstance(val, float) and val.is_integer():
            best_params[key] = int(val)
    return best_params


In [ ]:
"""
def xgb_eval(learning_rate, max_depth, gamma, colsample_bytree):
    params = {
        'learning_rate': learning_rate,
        'max_depth': int(max_depth),
        'gamma': gamma,
        'colsample_bytree': colsample_bytree
    }
    clf = XGBClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = f1_score(y_test, y_pred,average='macro')
    return score

bounds_xgboost = {
    'learning_rate': (0.01, 0.3),
    'max_depth': (3, 10),
    'gamma': (0, 1),
    'colsample_bytree': (0.5, 1)
}
best_params_xgboost = optimize_model(xgb_eval, bounds_xgboost)
best_params_xgboost['max_depth'] = int(best_params_xgboost['max_depth'])

xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)

"""
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)

In [ ]:
"""
def lgbm_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
    params = {
        'num_leaves': int(num_leaves),
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'max_depth': int(max_depth),
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'min_split_gain': min_split_gain,
        'min_child_weight': min_child_weight,
    }
    clf = LGBMClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = f1_score(y_test, y_pred,average='macro')
    return score

bounds_lgbm = {
    'num_leaves': (31, 500),
    'feature_fraction': (0.1, 0.9),
    'bagging_fraction': (0.8, 1),
    'max_depth': (5, 8.99),
    'lambda_l1': (0, 5),
    'lambda_l2': (0, 3),
    'min_split_gain': (0.001, 0.1),
    'min_child_weight': (5, 50),
}
best_params_lgbm = optimize_model(lgbm_eval, bounds_lgbm)
best_params_lgbm['max_depth'] = int(best_params_lgbm['max_depth'])
best_params_lgbm['num_leaves'] = int(best_params_lgbm['num_leaves'])

lgbm_model = LGBMRegressor()
lgbm_model.fit(X_train, y_train)
lgbm_predictions = lgbm_model.predict(X_test)
"""



lgbm_model = LGBMClassifier()
lgbm_model.fit(X_train, y_train)
lgbm_predictions = lgbm_model.predict(X_test)

In [ ]:

print("XGBoost Accuracy:", accuracy_score(y_test, xgb_predictions))
print("XGBoost F1 Score:", f1_score(y_test, xgb_predictions, average='macro'))
print("LGBM Accuracy:", accuracy_score(y_test, lgbm_predictions))
print("LGBM F1 Score:", f1_score(y_test, lgbm_predictions, average='macro'))




In [ ]:
"""
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt

xgb_r2 = r2_score(y_test, xgb_predictions)
xgb_rmse = sqrt(mean_squared_error(y_test, xgb_predictions))
print("XGBoost R^2:", xgb_r2)
print("XGBoost RMSE:", xgb_rmse)

lgbm_r2 = r2_score(y_test, lgbm_predictions)
lgbm_rmse = sqrt(mean_squared_error(y_test, lgbm_predictions))
print("LGBM R^2:", lgbm_r2)
print("LGBM RMSE:", lgbm_rmse)
"""

In [ ]:
import numpy as np


y_test_1d = np.ravel(y_test)
xgb_predictions_1d = np.ravel(xgb_predictions)
lgbm_predictions_1d = np.ravel(lgbm_predictions)

xgb_results = pd.DataFrame({'True_Values': y_test_1d, 'XGB_Predictions': xgb_predictions_1d})
lgbm_results = pd.DataFrame({'True_Values': y_test_1d, 'LGBM_Predictions': lgbm_predictions_1d})


xgb_results.to_csv('/content/drive/MyDrive/Colab Notebooks/HackED_beta/xgb_predictions.csv', index=False)
lgbm_results.to_csv('/content/drive/MyDrive/Colab Notebooks/HackED_beta/lgbm_predictions.csv', index=False)



In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

xgb_conf_matrix = confusion_matrix(y_test, xgb_predictions)
sns.heatmap(xgb_conf_matrix, annot=True, fmt='g')
plt.title('XGBoost Confusion Matrix')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.show()


lgbm_conf_matrix = confusion_matrix(y_test, lgbm_predictions)
sns.heatmap(lgbm_conf_matrix, annot=True, fmt='g')
plt.title('LGBM Confusion Matrix')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.show()


In [ ]:
import pickle
from joblib import dump

directory_path = '/content/drive/MyDrive/Colab Notebooks/HackED_beta'


with open(f'{directory_path}/xgb_model.pkl', 'wb') as file:
    pickle.dump(xgb_model, file)


with open(f'{directory_path}/lgbm_model.pkl', 'wb') as file:
    pickle.dump(lgbm_model, file)


dump(xgb_model, f'{directory_path}/xgb_model.joblib')
dump(lgbm_model, f'{directory_path}/lgbm_model.joblib')
